In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model

import csv
import pandas as pd
import math
%pylab inline

### Load in desired data from master CSVs.

In [ ]:
ELECTION_YEAR = 2018
INCUMBENT_YEAR = 2016

In [ ]:
ddata = pd.read_csv('undemocracy/data/demographics.csv', engine='python')
vdata_df = pd.read_csv('undemocracy/data/1976-2020-house.csv', engine='python')

In [ ]:
# only include desired election and incumbent years
vdata_inc = vdata_df[vdata_df['year'] == INCUMBENT_YEAR]
vdata = vdata_df[vdata_df['year'] == ELECTION_YEAR]

In [ ]:
# create congressional district id for vdata
vdata_inc['colid'] = vdata_inc['state_po'] + vdata_inc['district'].apply(str)
vdata['colid'] = vdata['state_po'] + vdata['district'].apply(str)

In [ ]:
# only include valid party affiliations
vdata_inc = vdata_inc[vdata_inc['party'].notna()]
vdata = vdata[vdata['party'].notna()]

In [ ]:
vdata_inc.reset_index(inplace = True, drop = True)
vdata.reset_index(inplace = True, drop = True)

### Determine incumbent party in each district from previous election cycle. 

In [ ]:
vdata_inc['percent votes'] = vdata_inc['candidatevotes'] / vdata_inc['totalvotes']
vdata['percent votes'] = vdata['candidatevotes'] / vdata['totalvotes']

In [ ]:
def is_incumbent(row):
    colid = row['colid']
    p_votes = row['percent votes']
    id_filter = vdata_inc[vdata_inc['colid'] == colid]
    if p_votes == id_filter['percent votes'].max() or math.isnan(p_votes):
        return 1
    else:
        return 0

vdata_inc['winner'] = vdata_inc.apply(is_incumbent, axis = 1)

In [ ]:
inc_map_dem = {}
inc_map_rep = {}
for index, row in vdata_inc.iterrows():
    if row['winner'] == 1:
        if row['party'] == 'DEMOCRAT':
            inc_map_dem[row['colid']] = 1
            inc_map_rep[row['colid']] = 0
        elif row['party'] == 'REPUBLICAN':
            inc_map_dem[row['colid']] = 0
            inc_map_rep[row['colid']] = 1

In [ ]:
# add incumbent information to the current election year's dataframe
def is_dem(row):
    if row['party'] == 'DEMOCRAT':
        return 1
    else:
        return 0

def is_rep(row):
    if row['party'] == 'REPUBLICAN':
        return 1
    else:
        return 0

vdata['party_democrat'] = vdata.apply(is_dem, axis = 1)
vdata['party_republican'] = vdata.apply(is_rep, axis = 1)
vdata['inc_democrat'] = (vdata['colid'].map(inc_map_dem)).astype('int')
vdata['inc_republican'] = (vdata['colid'].map(inc_map_rep)).astype('int')

In [ ]:
# dropping NaNs and getting the number of district
ddata = ddata.dropna()

#getting district number
ddata['district'] = ddata['State and District'].str.split(expand = True)[2]

#making congressional district id for ddata
ddata['colid'] = ddata['CDID'] + ddata['district'].apply(str)

### Determine winners per district for current year's election cycle. 

In [ ]:
# largest vote share per district 
def is_winner(row):
    colid = row['colid']
    p_votes = row['percent votes']
    id_filter = vdata[vdata['colid'] == colid]
    if p_votes == id_filter['percent votes'].max():
        return 1
    else:
        return 0
vdata['winner'] = vdata.apply(is_winner, axis = 1)

### Build new master CSV with incumbent & demographic information. 

In [ ]:
# merging data
new_data = ddata.merge(vdata, on=['colid'])

In [ ]:
# standardize numerical data
transform_data = ['Median Age', 'White', 'Black', 'AI/AN', 'PI', 'Other', 'Two Plus', 'Emp-LF Ratio', 'High School', 'Bachelor', 'Advanced', '% households w/ seniors', '% poverty']
min_max_scaler = MinMaxScaler()
new_data[transform_data] = min_max_scaler.fit_transform(new_data[transform_data])

In [ ]:
new_data.to_csv('undemocracy/data/new_data.csv')